<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/03_predict_stores_avg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

fatal: destination path 'Walmart-Recruiting-Store-Sales-Forecasting' already exists and is not an empty directory.
/content/Walmart-Recruiting-Store-Sales-Forecasting


In [ ]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

Setting up Kaggle credentials...
Ensuring data directory exists at 'data/'...
Data already exists at 'data/'. Skipping download and extraction.


# Load and Process Data

In [ ]:
from src import data_loader, processing
import importlib
importlib.reload(processing)

dataframes = data_loader.load_raw_data()
df = processing.create_store_level_data(dataframes, process_test=True)['train']
X_train, y_train, X_valid, y_valid = processing.split_data(df, separate_target=True)

print(f"Shapes of X_train and y_train: {X_train.shape}, {y_train.shape}")
print(f"Shapes of X_valid and y_valid: {X_valid.shape}, {y_valid.shape}")


del df, dataframes

Data loading complete.
Shapes of X_train and y_train: (4275, 14), (4275,)
Shapes of X_valid and y_valid: (2160, 14), (2160,)


In [ ]:
X_train

,Store,Date,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,2010-02-05,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,2,2010-02-05,False,A,202307,40.19,2.572,NaN,NaN,NaN,NaN,NaN,210.752605,8.324
2,3,2010-02-05,False,B,37392,45.71,2.572,NaN,NaN,NaN,NaN,NaN,214.424881,7.368
3,4,2010-02-05,False,A,205863,43.76,2.598,NaN,NaN,NaN,NaN,NaN,126.442065,8.623
4,5,2010-02-05,False,B,34875,39.70,2.572,NaN,NaN,NaN,NaN,NaN,211.653972,6.566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4270,41,2011-11-25,True,A,196321,36.37,3.424,256.95,1053.98,79621.20,40.76,2616.56,195.770400,6.759
4271,42,2011-11-25,True,C,39690,60.68,3.760,25.80,NaN,4.70,NaN,1932.72,129.836400,7.874
4272,43,2011-11-25,True,C,41062,55.70,3.236,3.37,NaN,5.76,NaN,212.75,210.088857,10.148
4273,44,2011-11-25,True,C,39910,38.89,3.445,NaN,NaN,NaN,NaN,256.45,129.836400,6.078


Learn strength of Store using XGboost.

In [ ]:
from feature_engineering import time_features, feature_transformers

import importlib
importlib.reload(time_features)

params = {
    'add_week_num' : True,
    'add_holiday_flags' : True,
    'add_holiday_proximity': True,
    'add_holiday_windows': True,
    'add_fourier_features': True,
    'add_month_and_year': True,
    'replace_time_index': True,
}

feature_adder = time_features.FeatureAdder(**params)
X_train_t = feature_adder.fit_transform(X_train)
X_valid_t = feature_adder.transform(X_valid)

object_transformer = feature_transformers.ObjectToCategory()
X_train_t = object_transformer.fit_transform(X_train_t)
X_valid_t = object_transformer.transform(X_valid_t)

store_income = y_train.groupby(X_train['Store']).mean().rename('total_store_income')
X_train_t = X_train_t.merge(store_income, on='Store', how='left')
X_valid_t = X_valid_t.merge(store_income, on='Store', how='left')

X_train_t['Store'] = X_train_t['Store'].astype('category')
X_valid_t['Store'] = X_valid_t['Store'].astype('category')

X_train_t = X_train_t.drop(columns=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'])
X_valid_t = X_valid_t.drop(columns=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'])

X_valid_t

,Store,IsHoliday,Type,Size,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,...,Days_until_next_SuperBowl,Days_since_last_SuperBowl,Days_until_next_Christmas,Days_since_last_Christmas,Days_until_next_Thanksgiving,Days_since_last_Thanksgiving,Days_until_next_LaborDay,Days_since_last_LaborDay,Date,total_store_income
0,1,False,A,151315,48.91,3.172,218.714733,7.866,12,2011,...,70,294,28,336,357,7,280,84,95,21335.179178
1,2,False,A,202307,48.74,3.172,218.359032,7.441,12,2011,...,70,294,28,336,357,7,280,84,95,26802.209424
2,3,False,B,37392,54.97,3.172,222.158952,7.197,12,2011,...,70,294,28,336,357,7,280,84,95,6205.055914
3,4,False,A,205863,38.71,3.176,129.845967,5.143,12,2011,...,70,294,28,336,357,7,280,84,95,28421.589498
4,5,False,B,34875,51.14,3.172,219.291729,6.300,12,2011,...,70,294,28,336,357,7,280,84,95,4918.866833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,41,False,A,196321,41.80,3.686,199.219532,6.195,10,2012,...,105,259,63,301,28,336,315,49,142,17500.927810
2156,42,False,C,39690,70.50,4.301,131.193097,6.943,10,2012,...,105,259,63,301,28,336,315,49,142,11525.665221
2157,43,False,C,41062,69.17,3.506,214.741539,8.839,10,2012,...,105,259,63,301,28,336,315,49,142,13496.818912
2158,44,False,C,39910,46.97,3.755,131.193097,5.217,10,2012,...,105,259,63,301,28,336,315,49,142,5871.945938


In [ ]:
from xgboost import XGBRegressor
from src.utils import wmae
from src.cross_validation import manual_model_search

model = XGBRegressor(
    objective='reg:squarederror',
    enable_categorical=True,
    random_state=42
)

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
}


metric_kwargs = {
    'is_holiday': X_valid_t['IsHoliday']
}

best_model, best_params, best_score = manual_model_search(
    model=model,
    param_grid=param_grid,
    X_train=X_train_t,
    y_train=y_train,
    X_valid=X_valid_t,
    y_valid=y_valid,
    metric_func=wmae,
    metric_kwargs=metric_kwargs
)

print("\nBest Params:", best_params)
print("Best Validation Score:", best_score)

Params: {'n_estimators': 50, 'learning_rate': 0.01, 'max_depth': 3} -> Score: 3822.3421
Params: {'n_estimators': 50, 'learning_rate': 0.01, 'max_depth': 5} -> Score: 3838.1085
Params: {'n_estimators': 50, 'learning_rate': 0.01, 'max_depth': 7} -> Score: 3829.3878
Params: {'n_estimators': 50, 'learning_rate': 0.05, 'max_depth': 3} -> Score: 1382.2604
Params: {'n_estimators': 50, 'learning_rate': 0.05, 'max_depth': 5} -> Score: 1182.9708
Params: {'n_estimators': 50, 'learning_rate': 0.05, 'max_depth': 7} -> Score: 1125.7558
Params: {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 3} -> Score: 1085.3248
Params: {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 5} -> Score: 951.3106
Params: {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 7} -> Score: 886.5294
Params: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 3} -> Score: 2577.2952
Params: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 5} -> Score: 2548.0074
Params: {'n_estimators': 100, 'lear

In [ ]:
from xgboost import XGBRegressor
from src.utils import wmae
from src.cross_validation import manual_model_search

model = XGBRegressor(
    objective='reg:squarederror',
    enable_categorical=True,
    random_state=42,
    n_estimators=200,
    learning_rate=0.1,
    max_depth=7,
)

param_grid = {
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'min_child_weight': [1, 5, 10]
}

metric_kwargs = {
    'is_holiday': X_valid_t['IsHoliday']
}

best_model, best_params, best_score = manual_model_search(
    model=model,
    param_grid=param_grid,
    X_train=X_train_t,
    y_train=y_train,
    X_valid=X_valid_t,
    y_valid=y_valid,
    metric_func=wmae,
    metric_kwargs=metric_kwargs
)

print("\nBest Params:", best_params)
print("Best Validation Score:", best_score)

Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 1} -> Score: 912.9621
Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 5} -> Score: 875.1097
Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 10} -> Score: 873.6118
Params: {'subsample': 0.6, 'colsample_bytree': 0.7, 'min_child_weight': 1} -> Score: 834.3131
Params: {'subsample': 0.6, 'colsample_bytree': 0.7, 'min_child_weight': 5} -> Score: 796.6159
Params: {'subsample': 0.6, 'colsample_bytree': 0.7, 'min_child_weight': 10} -> Score: 797.7321
Params: {'subsample': 0.6, 'colsample_bytree': 1.0, 'min_child_weight': 1} -> Score: 862.0064
Params: {'subsample': 0.6, 'colsample_bytree': 1.0, 'min_child_weight': 5} -> Score: 816.0036
Params: {'subsample': 0.6, 'colsample_bytree': 1.0, 'min_child_weight': 10} -> Score: 794.0671
Params: {'subsample': 0.8, 'colsample_bytree': 0.5, 'min_child_weight': 1} -> Score: 895.2297
Params: {'subsample': 0.8, 'colsample_bytree': 0.5, 'min_

In [ ]:
from re import sub
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from src.utils import wmae as compute_wmae

model = XGBRegressor(
    objective='reg:squarederror',
    enable_categorical=True,
    random_state=42,
    n_estimators=200,
    learning_rate=0.1,
    max_depth=7,
    subsample=0.6,
    colsample_bytree=1.0,
    min_child_weight=10
)

model.fit(X_train_t, y_train)
train_preds = model.predict(X_train_t)
valid_preds = model.predict(X_valid_t)

wma = compute_wmae(y_valid, valid_preds, is_holiday=X_valid_t['IsHoliday'])
print(f"Validation WMAE: {wma:.2f}")
mae = mean_absolute_error(y_valid, valid_preds)
print(f"Validation MAE: {mae:.2f}")

wma = compute_wmae(y_train, train_preds, is_holiday=X_train_t['IsHoliday'])
print(f"Train WMAE: {wma:.2f}")
mae = mean_absolute_error(y_train, train_preds)
print(f"Train MAE: {mae:.2f}")


Validation WMAE: 794.07
Validation MAE: 791.71
Train WMAE: 334.37
Train MAE: 323.32


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore", message="No frequency information was provided")


df_residuals = X_train[['Store', 'Date']].copy()
residuals = y_train - train_preds
df_residuals['Residual'] = residuals.values

arima_models = {}
store_forecasts = {}
n_forecast_steps = len(y_valid)  # or any steps you want

for store_id, group in df_residuals.groupby('Store'):
    ts = group.set_index('Date')['Residual']
    order = (1,1,0)
    print(f"Store {store_id} ARIMA order: {order}")
    model = ARIMA(ts, order=order)
    model_fit = model.fit()
    arima_models[store_id] = model_fit

    forecast = model_fit.forecast(steps=n_forecast_steps)
    store_forecasts[store_id] = forecast

Store 1 ARIMA order: (1, 1, 0)
Store 2 ARIMA order: (1, 1, 0)
Store 3 ARIMA order: (1, 1, 0)
Store 4 ARIMA order: (1, 1, 0)
Store 5 ARIMA order: (1, 1, 0)
Store 6 ARIMA order: (1, 1, 0)
Store 7 ARIMA order: (1, 1, 0)
Store 8 ARIMA order: (1, 1, 0)
Store 9 ARIMA order: (1, 1, 0)
Store 10 ARIMA order: (1, 1, 0)
Store 11 ARIMA order: (1, 1, 0)
Store 12 ARIMA order: (1, 1, 0)
Store 13 ARIMA order: (1, 1, 0)
Store 14 ARIMA order: (1, 1, 0)
Store 15 ARIMA order: (1, 1, 0)
Store 16 ARIMA order: (1, 1, 0)
Store 17 ARIMA order: (1, 1, 0)
Store 18 ARIMA order: (1, 1, 0)
Store 19 ARIMA order: (1, 1, 0)
Store 20 ARIMA order: (1, 1, 0)
Store 21 ARIMA order: (1, 1, 0)
Store 22 ARIMA order: (1, 1, 0)
Store 23 ARIMA order: (1, 1, 0)
Store 24 ARIMA order: (1, 1, 0)
Store 25 ARIMA order: (1, 1, 0)
Store 26 ARIMA order: (1, 1, 0)
Store 27 ARIMA order: (1, 1, 0)
Store 28 ARIMA order: (1, 1, 0)
Store 29 ARIMA order: (1, 1, 0)
Store 30 ARIMA order: (1, 1, 0)
Store 31 ARIMA order: (1, 1, 0)
Store 32 ARIMA or

In [ ]:
import pandas as pd

def add_arima_residuals(X, y_pred, arima_models):
    X_ = X.copy()
    X_['Original_Pred'] = y_pred
    X_['ARIMA_Residual'] = 0.0

    for store_id, group in X.groupby('Store'):
        model = arima_models.get(store_id)
        if model is None:
            continue
        n = len(group)
        resid_forecast = model.forecast(steps=n)
        X_.loc[group.index, 'ARIMA_Residual'] = resid_forecast.values[:n]
    final_pred = X_['Original_Pred'] + X_['ARIMA_Residual']
    return final_pred

valid_final = add_arima_residuals(X_valid_t, valid_preds, arima_models)
wma = compute_wmae(y_valid, valid_final, is_holiday=X_valid_t['IsHoliday'])
print(f"Validation WMAE: {wma:.2f}")
mae = mean_absolute_error(y_valid, valid_final)
print(f"Validation MAE: {mae:.2f}")

train_filan = add_arima_residuals(X_train_t, train_preds, arima_models)
wma = compute_wmae(y_train, train_filan, is_holiday=X_train_t['IsHoliday'])
print(f"Train WMAE: {wma:.2f}")
mae = mean_absolute_error(y_train, train_filan)
print(f"Train MAE: {mae:.2f}")

/tmp/ipython-input-102-1582318855.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for store_id, group in X.groupby('Store'):


Validation WMAE: 805.23
Validation MAE: 801.05


/tmp/ipython-input-102-1582318855.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for store_id, group in X.groupby('Store'):


Train WMAE: 415.50
Train MAE: 415.47


# Predict Test

In [ ]:
from feature_engineering import time_features, feature_transformers

import importlib
importlib.reload(time_features)

params = {
    'add_week_num' : True,
    'add_holiday_flags' : True,
    'add_holiday_proximity': False,
    'add_holiday_windows': False,
    'add_fourier_features': False,
    'add_month_and_year': True,
    'list_of_holiday_proximity': [],
    'replace_time_index': True,
}


X_test = dt['test'].copy()

dates = X_test['Date']

X_test = feature_adder.transform(X_test)

X_test = object_transformer.transform(X_test)

X_test = X_test.merge(store_income, on='Store', how='left')

X_test

In [ ]:
import pandas as pd
from IPython.display import FileLink



test_preds = best_model.predict(X_test)

X_test = X_test.copy()
X_test['Stor_Dept_time'] = (
    X_test['Store'].astype(str) + '_' +
    X_test['Dept'].astype(str) + '_' +
    dates.astype(str)
)

# Prepare submission
submission = pd.DataFrame({
    'Id': X_test['Stor_Dept_time'],
    'Weekly_Sales': test_preds
})

# Save to CSV
submission_path = 'submission.csv'
submission.to_csv(submission_path, index=False)

print(f"Submission file saved: {submission_path}")

# Display download link
display(FileLink(submission_path))
